In [37]:
import warnings
import pandas as pd
import numpy as np
from pathlib import Path

from src.data.raw.cleaner import clean_descriptions

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import RSLPStemmer
from scipy.sparse import hstack
# from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

## Data exploration

In [2]:
expenses = pd.read_csv(
    Path.cwd() / "database" / "all_expenses.csv",
    dtype={
        "amount": np.float32,
        "year": np.int32,
        "month": np.int8,
        "recurrent": np.int8,
        "description": str,
        "cleaned_description": str,
        "category": str,
        "subcategory": str,
        "bank": str,
    },
)
expenses.tail()

,description,amount,category,subcategory,year,month,recurrent,bank,cleaned_description
1997,CREDITO OPERACAO B3-26/09/2024,0.360000,Poupança/Dívidas/Investimentos,Investimentos,2024,9,0,C6,NaN
1998,PGTO DE BOLETO-Boleto,234.490005,Transporte,Seguro do automóvel,2024,9,1,C6,NaN
1999,TRANSF ENVIADA PIX,25.000000,Diversos,Despesas Diversas,2024,9,0,C6,NaN
2000,TRANSF ENVIADA PIX,5.000000,Diversos,Despesas Diversas,2024,9,0,C6,NaN
2001,Fatura cartão Inter,306.899994,Diversos,Despesas Diversas,2024,9,1,Inter,NaN


In [3]:
expenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002 entries, 0 to 2001
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   description          2002 non-null   object 
 1   amount               2002 non-null   float32
 2   category             2002 non-null   object 
 3   subcategory          2002 non-null   object 
 4   year                 2002 non-null   int32  
 5   month                2002 non-null   int8   
 6   recurrent            2002 non-null   int8   
 7   bank                 2002 non-null   object 
 8   cleaned_description  1437 non-null   object 
dtypes: float32(1), int32(1), int8(2), object(5)
memory usage: 97.9+ KB


In [4]:
expenses.loc[:, "amount"] = expenses["amount"].apply(lambda x: round(x, 2))

In [5]:
expenses.loc[expenses["cleaned_description"].isna(), "cleaned_description"] = expenses.loc[expenses["cleaned_description"].isna(), "description"]

In [6]:
expenses_cleaned = clean_descriptions([], expenses)

In [7]:
expenses_cleaned[["description", "cleaned_description"]].sample(10)

,description,cleaned_description
1695,PGTO DE BOLETO-Boleto,bolet
952,PAPUERA BAR,papu bar
1346,PAG*LOJASRENNERSA,lojasrenners
496,Fatura cartão Inter,fatur int
504,Transferência enviada pelo Pix - POPO'S DOG HO...,pop dog hotel itau unibanc agenc
1643,99* POP 23MAI 13H00MIN,pop
306,SUPERMERCADO GUANABARA,supermerc guanab
847,FABRICA ROCA,fabr roc
968,FARMACIA SAO JOAO,farmac sao joa
1098,PAG*LOJASRENNERSA,lojasrenners


In [9]:
expenses_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002 entries, 0 to 2001
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   description          2002 non-null   object 
 1   amount               2002 non-null   float64
 2   category             2002 non-null   object 
 3   subcategory          2002 non-null   object 
 4   year                 2002 non-null   int32  
 5   month                2002 non-null   int8   
 6   recurrent            2002 non-null   int8   
 7   bank                 2002 non-null   object 
 8   cleaned_description  2002 non-null   object 
dtypes: float64(1), int32(1), int8(2), object(5)
memory usage: 105.7+ KB


In [13]:
expenses_cleaned.groupby("subcategory").count()

,description,amount,category,year,month,recurrent,bank,cleaned_description
subcategory,,,,,,,,
Acomodação,2,2,2,2,2,2,2,2
Animais de estimação,81,81,81,81,81,81,81,81
Assinaturas,42,42,42,42,42,42,42,42
Bares/Pubs,35,35,35,35,35,35,35,35
Beleza e Cuidados Pessoais,41,41,41,41,41,41,41,41
Celular/Telefone Móvel,51,51,51,51,51,51,51,51
Comer Fora,143,143,143,143,143,143,143,143
Compartilhamento de Carro,17,17,17,17,17,17,17,17
Compras para casa,321,321,321,321,321,321,321,321


In [14]:
expenses_cleaned.loc[expenses_cleaned["subcategory"] == "Transporte", "subcategory"] = "Despesas Diversas"
expenses_cleaned.loc[expenses_cleaned["subcategory"] == "subcategory.lodging_travel", "subcategory"] = "Despesas Diversas"

In [16]:
import plotly.express as px

px.histogram(
    expenses_cleaned, 
    x="category",
    color="category",
)

In [17]:
px.histogram(
    expenses_cleaned,
    x="subcategory",
    color="category",
)

In [19]:
def run_models(models, X_train, X_test, y_train, y_test):
    results = {}
    
    for name, model in models.items():
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),  
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        
        results[name] = {
            "accuracy": accuracy,
            "classification_report": report
        }
        print(f"Modelo: {name}")
        print(f"Acurácia: {accuracy*100:.2f}%")
        print(report)
    
    return results

In [36]:
df = expenses_cleaned.copy()

models = {
    "RandomForest": RandomForestClassifier(class_weight="balanced"),
    "SVM": SVC(class_weight="balanced"),
    "MLPClassifier": MLPClassifier(max_iter=500),
}

X = df['cleaned_description']
y = df['subcategory']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

results = run_models(models, X_train, X_test, y_train, y_test)

Modelo: RandomForest
Acurácia: 91.68%
                                   precision    recall  f1-score   support

                       Acomodação       1.00      1.00      1.00         1
             Animais de estimação       0.95      0.79      0.86        24
                      Assinaturas       1.00      0.92      0.96        13
                       Bares/Pubs       1.00      0.91      0.95        11
       Beleza e Cuidados Pessoais       0.83      0.83      0.83        12
           Celular/Telefone Móvel       1.00      1.00      1.00        15
                       Comer Fora       0.94      0.77      0.85        43
        Compartilhamento de Carro       1.00      1.00      1.00         5
                Compras para casa       0.98      0.99      0.98        96
                Despesas Diversas       0.74      0.88      0.81        92
                 Despesas Médicas       1.00      1.00      1.00         2
                     Eletricidade       1.00      0.75      0

# Feature Engineering

## With keywords as features

In [20]:
keyword_mapping: dict[str, list[str]] = {
    # Moradia
    "Hipoteca/Aluguel": ["hipoteca", "aluguel", "locação", "imóvel"],
    "Manutenção residencial": ["manutenção", "reparos", "conserto", "residencial"],
    "Impostos sobre a propriedade": ["iptu", "imposto", "propriedade"],
    "Compras para a casa": ["móveis", "decoração", "eletrodoméstico", "cama", "mesa", "banho"],
    
    # Transporte
    "Gasolina": ["gasolina", "combustível", "posto", "etanol", "diesel"],
    "Seguro do automóvel": ["seguro automóvel", "proteção veículo", "apólice automóvel"],
    "Manutenção do automóvel": ["revisão", "oficina", "mecânico", "troca de óleo", "pneus"],
    "Impostos e taxas do automóvel": ["ipva", "licenciamento", "detran", "multas"],
    "Transporte público": ["ônibus", "metrô", "bilhete", "passagem"],
    "Compartilhamento de carros": ["uber", "99", "cabify", "carona"],

    # Utilidades
    "Eletricidade": ["energia", "luz", "eletricidade", "enel", "ceee", "equatorial"],
    "Água": ["água", "sanasa", "sabesp", "sanep"],
    "Tarifas bancárias": ["tarifa bancária", "taxa banco", "cesta serviços"],
    "Celular": ["celular", "telefone", "tim", "claro", "vivo", "oi"],
    "Internet": ["internet", "wi-fi", "banda larga", "net", "fibra óptica"],

    # Alimentação
    "Mantimentos": ["supermercado", "mercado", "compras", "alimentos", "mantimentos", "macro atacado"],
    "Comer fora": ["restaurante", "lanchonete", "pizzaria", "bar", "café"],

    # Finanças
    "Poupança": ["poupança", "depósito", "reserva", "fundo"],
    "Dívidas": ["empréstimo", "financiamento", "dívida", "parcelamento"],
    "Investimentos": ["investimento", "ações", "bolsa", "renda fixa", "tesouro direto", "cdb", "rdb"],

    # Saúde
    "Plano de saúde": ["plano de saúde", "unimed", "bradesco saúde"],
    "Seguro saúde": ["seguro saúde", "apólice saúde", "cobertura médica"],
    "Despesas médicas": ["consultas", "exames", "tratamentos", "procedimentos médicos"],
    "Fitness": ["academia", "exercícios", "ginástica", "crossfit", "pilates"],
    "Medicações": ["remédio", "farmácia", "medicamento", "genérico"],
    "Suplementos": ["suplemento", "vitamina", "proteína", "whey"],

    # Pessoais
    "Vestuário": ["roupa", "calçado", "vestido", "camiseta", "sapato"],
    "Beleza e cuidados pessoais": ["cabeleireiro", "manicure", "barbearia", "cosméticos"],
    "Hobbies e recreações": ["livros", "videogame", "instrumentos", "brinquedos"],
    "Assinaturas": ["netflix", "spotify", "amazon prime", "hbo", "globo play", "investidor10"],
    "Estudos": ["curso", "escola", "faculdade", "matrícula", "aula", "seminário"],

    # Lazer
    "Filmes e séries": ["cinema", "filme", "série", "tv", "streaming"],
    "Eventos e atividades": ["evento", "concerto", "show", "festa", "balada"],
    "Bares e pubs": ["bar", "pub", "bebida", "cerveja", "whisky", "vinho"],

    # Viagens
    "Locomoção em viagens": ["passagem", "avião", "translado"],
    "Hospedagem": ["hotel", "airbnb", "pousada", "resort"],
    "Outros custos com viagens": ["souvenir", "passeios", "excursão", "tour", "guia"],

    # Diversos
    "Presentes e doações": ["presente", "doação", "aniversário", "casamento"],
    "Despesas diversas": ["diversos", "outros", "aleatórios", "extra"],
    "Férias": ["férias", "viagem", "descanso"],
    "Pets": ["pet", "animal", "ração", "veterinário", "banho e tosa", "vet"]
}

In [21]:
def map_keywords(description: str, keyword_mapping: dict[str, list[str]]) -> dict:
    """
    Maps keywords to relevant categories.
    
    Parameters:
    - description: Cleaned description from the dataframe.
    - keyword_mapping: A dictionary with subcategories as keys and lists of keywords as values.

    Returns:
    - A dictionary where the key is the subcategory and the value is 1 (if a keyword is present) or 0.
    """
    mapped_features = {category: 0 for category in keyword_mapping}
    
    for category, keywords in keyword_mapping.items():
        for keyword in keywords:
            if keyword in description:
                mapped_features[category] = 1
                break  # if it finds a word it stops for this category

    return mapped_features

def add_keyword_features(df: pd.DataFrame, keyword_mapping: dict[str, list[str]]) -> pd.DataFrame:
    """
    Adds keyword-based features to the DataFrame.
    
    Parameters:
    - df: DataFrame containing the cleaned descriptions.
    - keyword_mapping: A dictionary with subcategories as keys and lists of keywords as values.

    Returns:
    - The updated DataFrame with new columns for each subcategory.
    """
    keyword_features = df['cleaned_description'].apply(
        lambda desc: map_keywords(desc, keyword_mapping)
    )
    
    keyword_df = pd.DataFrame(keyword_features.tolist(), index=df.index)
    
    df = pd.concat([df, keyword_df], axis=1)
    
    return df

In [22]:
def add_keyword_features(df: pd.DataFrame, keyword_mapping: dict) -> pd.DataFrame:
    """
    Adds keyword-based features to the DataFrame.
    
    Parameters:
    - df: DataFrame containing the cleaned descriptions.
    - keyword_mapping: A dictionary with subcategories as keys and lists of keywords as values.

    Returns:
    - The updated DataFrame with new columns for each subcategory.
    """
    keyword_features = df['cleaned_description'].apply(
        lambda desc: map_keywords(desc, keyword_mapping)
    )
    
    keyword_df = pd.DataFrame(keyword_features.tolist(), index=df.index)
    
    df = pd.concat([df, keyword_df], axis=1)
    
    return df


In [34]:
stemmer = RSLPStemmer()

def stem_keywords(keyword_mapping):
    stemmed_mapping = {}
    for subcategory, keywords in keyword_mapping.items():
        stemmed_keywords = [stemmer.stem(keyword) for keyword in keywords]
        stemmed_mapping[subcategory] = stemmed_keywords
    return stemmed_mapping

stemmed_keyword_mapping = stem_keywords(keyword_mapping)

In [35]:
df = add_keyword_features(expenses_cleaned, keyword_mapping)

feature_columns = ["cleaned_description"] + list(keyword_mapping.keys())
X = df[feature_columns]
y = df['subcategory']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

for name, model in models.items():
    print(f"Training {name}...")

    pipeline = Pipeline([
        ('features', ColumnTransformer([
            ('text', TfidfVectorizer(), 'cleaned_description'),
        ])),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)

    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)

    print(f"Accuracy for {name}: {accuracy*100:.2f}%")
    print(f"Classification Report for {name}:\n{report}")



Training RandomForest...
Accuracy for RandomForest: 90.85%
Classification Report for RandomForest:
                                   precision    recall  f1-score   support

                       Acomodação       1.00      1.00      1.00         1
             Animais de estimação       0.95      0.79      0.86        24
                      Assinaturas       1.00      0.92      0.96        13
                       Bares/Pubs       1.00      0.91      0.95        11
       Beleza e Cuidados Pessoais       0.83      0.83      0.83        12
           Celular/Telefone Móvel       1.00      1.00      1.00        15
                       Comer Fora       0.89      0.74      0.81        43
        Compartilhamento de Carro       1.00      1.00      1.00         5
                Compras para casa       0.98      0.99      0.98        96
                Despesas Diversas       0.72      0.88      0.79        92
                 Despesas Médicas       1.00      1.00      1.00         2
